In [ ]:
# interact with LLM via API authn

import os
from openai import OpenAI
from dotenv import load_dotenv


load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


try:
    # This will tell you if your API key is valid and show account info
    models = client.models.list()
    print(f"Available models: {len(models.data)}")

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful DevOps assistant."},
            {"role": "user", "content": "Explain how CI/CD works in Azure DevOps."}
        ]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(e)

In [ ]:
# plot sigmoid function

import numpy as np
import matplotlib.pyplot as plt

# Define the function
x = np.linspace(-10, 10, 400)
y = 1 / (1 + np.exp(-x))

# Plot
plt.figure(figsize=(8, 5))
plt.plot(x, y, linewidth=2)
plt.title(r"Plot of $f(x) = \frac{1}{1 + e^{-x}}$ (Sigmoid Function)")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.grid(True)
plt.show()

In [ ]:
# Fine-tuning

from transformers import pipeline

pipe = pipeline('text2text-generation', model='google/flan-t5-base')

prompt = "Mark: Hello mr. Moh \n Mark: Hope you're doing well? \n Moh: Meow"
